<h1><B><U>Final Project 1</U></B></h1>

<h2><B>Text Language Detection</B></h2>

<h3>Mohamad Javad Sahebnasi / 97243043


<h2><B>#1: <u>dataset</u></B></h2>
<h1><B>Solution #1:</B></h1>
<h3>TF-IDF / CountVec + some models</h3>
<h2><B>#2: <u>pre-processing</u></B></h2>
<h2><B>#3: <u>preparing test data</u></B></h2>
<h2><B>#4.0: <u>my KNN implementation</u></B></h2>
<h2><B>#4: <u>model & prediction</u></B></h2>
<h2><B>#5: <u>output</u></B></h2>
<h1><B>Solution #2:</B></h1>
<h3>TF-IDF + a new feature + SVM</h3>

In [1]:
!pip install bottleneck

     |████████████████████████████████| 88 kB 706 kB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for bottleneck: filename=Bottleneck-1.3.2-cp37-cp37m-linux_x86_64.whl size=334959 sha256=b9dc90a781f8e8b315f48606bc21ec6c4c5251d156d49d7fee904d3c048c6df9
  Stored in directory: /root/.cache/pip/wheels/87/85/9c/a325c89ff0498660ef8a335fb4b3912939c273ea4f094af29f
Successfully built bottleneck


In [2]:
import numpy as np
import pandas as pd

import gc
import time

# pre-process:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

# models:
from sklearn import linear_model
from sklearn import neighbors
from sklearn import svm

# KNN implementation
from numpy import linalg
import bottleneck as bn
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [3]:
# here it must be "Pushto" instead of "Pashto" :|
languages = ['Persian','Arabic','Pushto','English','German','French']

<h2><B>#1: <u>dataset</u></B></h2>
I'm using 2 datasets from:

https://www.kaggle.com/zarajamshaid/language-identification-datasst</br>
https://www.kaggle.com/basilb2s/language-detection

In [4]:
raw_train_1 = pd.read_csv('../input/language-identification-datasst/dataset.csv')
raw_train_2 = pd.read_csv('../input/language-detection/Language Detection.csv')

raw_train_1.columns = raw_train_2.columns

In [5]:
train = pd.concat([raw_train_1[raw_train_1['Language'].isin(languages)], raw_train_2[raw_train_2['Language'] == 'German']])
train = train.reset_index(drop=True)

In [6]:
# changing "Pushto" to "Pashto" :|
train[train['Language'] == 'Pushto'] = train[train['Language'] == 'Pushto'].replace(['Pushto'], 'Pashto')

In [7]:
set(train['Language'])

{'Arabic', 'English', 'French', 'German', 'Pashto', 'Persian'}

<h1><B>Solution #1:</B></h1>

<h2><B>#2: <u>pre-processing</u></B></h2>

In [8]:
def simple_preProcess(X, code):
    
    if code == 'tfidf':
        vectorizer = TfidfVectorizer()
    elif code == 'countVec':
        vectorizer = CountVectorizer()
    else:
        raise ValueError('invalid value for code')
        
    df = pd.DataFrame.sparse.from_spmatrix(vectorizer.fit_transform(X))

    return df, vectorizer.get_feature_names()

In [9]:
#both create/return the same dictionary
X_tfidf, dictionary = simple_preProcess(train['Text'], 'tfidf')
X_countVec, dictionary = simple_preProcess(train['Text'], 'countVec')

In [10]:
Y = train['Language']

<h2><B>#3: <u>preparing test data</u></B></h2>


In [11]:
def simple_test_preprocess(T, dictionary, code):
    
    if code == 'tfidf':
        vectorizer = TfidfVectorizer(vocabulary = dictionary)
    elif code == 'countVec':
        vectorizer = CountVectorizer(vocabulary = dictionary)
    else:
        raise ValueError('invalid value for code')

    df = pd.DataFrame.sparse.from_spmatrix(vectorizer.fit_transform(T))

    return df

In [12]:
test = pd.read_csv('../input/ai-course-2021-sbu/task1 - task1.csv')

In [13]:
test.head()

,Id
0,واتهمت وزارة العدل الأمركية جون مكافي بالتهرب ...
1,Eine propositionale Einstellung (englisch prop...
2,Rule-based machine learning (RBML) is a term i...
3,"In a genetic algorithm, a population of candid..."
4,Its fields can be divided into theoretical and...


In [45]:
tfidf_preprocessed_test = simple_test_preprocess(test['Id'], dictionary, 'tfidf')
countVec_preprocessed_test = simple_test_preprocess(test['Id'], dictionary, 'countVec')

<h2><B>#4.0: <u>my KNN implementation</u></B></h2>

In [15]:
def euclidean_distance(X1, X2):
    return linalg.norm(X2-X1, axis=1)

In [16]:
def find_nearest_values(max_k, X_dataset, X_input):
    (samples_num, features_num) = X_dataset.shape
    tests_num = len(X_input)

    result = []

    for test_index in range(tests_num):
        
        distances = euclidean_distance(np.array(X_input.iloc[test_index]), np.array(X_dataset))

        # pick up first <k> neighbours 
        distances = bn.argpartition(distances, kth = max_k)
        knn_indices = distances[:max_k+1]

        result.append(knn_indices)
        
    return result

In [17]:
max_k = 6

In [18]:
def my_KNN_validation(X_train, X_validation, Y, y_validation):
    
    strt_time = time.time()
    print('validating ...')
    knn_indeices = find_nearest_values(max_k, X_train, X_validation)
    print('validation time : ', time.time() - strt_time)

    best_accuracy = 0
    best_k = 3
    for k in range(3, max_k+1):
        y_predict = Y[[stats.mode(knn_indeices[i][:k])[0].flatten()[0] for i in range(len(knn_indeices))]]
        accuracy = accuracy_score(y_validation, y_predict)
        print('k: ', k, ' - accuracy: ', accuracy)
        
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_k = k

    print('best k: ', best_k, best_accuracy)
    return best_k

In [19]:
def my_KNN_predict(X_train, X_test, Y, best_k):
    
    knn_indeices = find_nearest_values(max_k, X_train, X_test)
    T = Y[[stats.mode(knn_indeices[i][:best_k])[0].flatten()[0] for i in range(len(knn_indeices))]]
    return T

<h2><b>validation & finding the best k:</b></h2>

will take a long time ...

In [ ]:
# X_new_train, X_validation, y_new_train, y_validation = train_test_split(X_tfidf, Y, test_size=0.10)

In [ ]:
# best_k = my_KNN_validation(X_new_train, X_validation, Y, y_validation)

<h2><B>#4.1: <u>model</u></B></h2>


<h2><B>#4.1.1: <u>tfidf</u></B></h2>

In [15]:
SGD_model_tfidf = linear_model.SGDClassifier()
KNN_model_tfidf = neighbors.KNeighborsClassifier(5)
linSVM_model_tfidf = svm.LinearSVC(C=0.1)

In [16]:
SGD_model_tfidf.fit(X_tfidf, Y)

SGDClassifier()

In [17]:
KNN_model_tfidf.fit(X_tfidf, Y)

KNeighborsClassifier()

In [18]:
linSVM_model_tfidf.fit(X_tfidf, Y)

LinearSVC(C=0.1)

<h2><B>#4.1.2: <u>CountVec</u></B></h2>

In [19]:
SGD_model_countVec = linear_model.SGDClassifier()
KNN_model_countVec = neighbors.KNeighborsClassifier(5)
linSVM_model_countVec = svm.LinearSVC(C=0.1)

In [20]:
SGD_model_countVec.fit(X_countVec, Y)

SGDClassifier()

In [21]:
KNN_model_countVec.fit(X_countVec, Y)

KNeighborsClassifier()

In [22]:
linSVM_model_countVec.fit(X_countVec, Y)

LinearSVC(C=0.1)

<h2><B>#4.2: <u>prediction</u></B></h2>

<h2><B>#4.2.1: <u>tfidf</u></B></h2>

In [23]:
SGD_prediction_tfidf = SGD_model_tfidf.predict(tfidf_preprocessed_test)

In [24]:
KNN_prediction_tfidf = KNN_model_tfidf.predict(tfidf_preprocessed_test)

In [25]:
linSVM_prediction_tfidf = linSVM_model_tfidf.predict(tfidf_preprocessed_test)

In [ ]:
# my_KNN_prediction_tfidf = my_KNN_predict(X_tfidf, tfidf_preprocessed_test, Y, 3)

<h2><B>#4.2.2: <u>CountVec</u></B></h2>

In [26]:
SGD_prediction_countVec = SGD_model_countVec.predict(countVec_preprocessed_test)

In [32]:
KNN_prediction_countVec = KNN_model_countVec.predict(countVec_preprocessed_test)

In [27]:
linSVM_prediction_countVec = linSVM_model_countVec.predict(countVec_preprocessed_test)

<h2><B>#5: <u>output</u></B></h2>

<h2><B>#5.1: <u>tfidf</u></B></h2>

In [34]:
SGD_output_tfidf = pd.DataFrame({'Id' : test['Id'], 'Category' : SGD_prediction_tfidf})

In [35]:
SGD_output_tfidf

,Id,Category
0,واتهمت وزارة العدل الأمركية جون مكافي بالتهرب ...,Arabic
1,Eine propositionale Einstellung (englisch prop...,German
2,Rule-based machine learning (RBML) is a term i...,English
3,"In a genetic algorithm, a population of candid...",English
4,Its fields can be divided into theoretical and...,English
...,...,...
95,اللُّغَة العَرَبِيّة هي أكثر اللغات السامية تح...,Arabic
96,ريتشارد ماثيو ستالمان (بالإنجليزية: Richard Ma...,Arabic
97,Bob Dylan (born Robert Allen Zimmerman; May 24...,English
98,بعضی از ثروتمندترین و معروفترین افراد دنیا همی...,Persian


In [36]:
SGD_output_tfidf.to_csv('./SGD_output_tfidf.csv', index=False)

In [37]:
KNN_output_tfidf = pd.DataFrame({'Id' : test['Id'], 'Category' : KNN_prediction_tfidf})

In [38]:
KNN_output_tfidf.to_csv('./KNN_output_tfidf.csv', index=False)

In [39]:
linSVM_output_tfidf = pd.DataFrame({'Id' : test['Id'], 'Category' : linSVM_prediction_tfidf})

In [40]:
linSVM_output_tfidf.to_csv('./linSVM_output_tfidf.csv', index=False)

In [ ]:
# my_KNN_T_tfidf = pd.DataFrame({'Category' : my_KNN_prediction_tfidf})
# my_KNN_T_tfidf = my_KNN_T_tfidf.reset_index(drop = True)

In [ ]:
# my_KNN_output_tfidf = pd.DataFrame({'Id' : test['Id'], 'Category' : my_KNN_T_tfidf['Category']})

In [ ]:
# my_KNN_output_tfidf.to_csv('./my_KNN_output_tfidf.csv', index=False)

<h2><B>#5.2: <u>countVec</u></B></h2>

In [41]:
SGD_output_countVec = pd.DataFrame({'Id' : test['Id'], 'Category' : SGD_prediction_countVec})

In [42]:
SGD_output_countVec

,Id,Category
0,واتهمت وزارة العدل الأمركية جون مكافي بالتهرب ...,Arabic
1,Eine propositionale Einstellung (englisch prop...,German
2,Rule-based machine learning (RBML) is a term i...,English
3,"In a genetic algorithm, a population of candid...",English
4,Its fields can be divided into theoretical and...,English
...,...,...
95,اللُّغَة العَرَبِيّة هي أكثر اللغات السامية تح...,Arabic
96,ريتشارد ماثيو ستالمان (بالإنجليزية: Richard Ma...,Arabic
97,Bob Dylan (born Robert Allen Zimmerman; May 24...,English
98,بعضی از ثروتمندترین و معروفترین افراد دنیا همی...,Persian


In [43]:
SGD_output_countVec.to_csv('./SGD_output_countVec.csv', index=False)

In [44]:
KNN_output_countVec = pd.DataFrame({'Id' : test['Id'], 'Category' : KNN_prediction_countVec})

In [45]:
KNN_output_countVec.to_csv('./KNN_output_countVec.csv', index=False)

In [46]:
linSVM_output_countVec = pd.DataFrame({'Id' : test['Id'], 'Category' : linSVM_prediction_countVec})

In [47]:
linSVM_output_countVec.to_csv('./linSVM_output_countVec.csv', index=False)

<h1><B>Solution #2:</B></h1>

In [28]:
persian_specs = {'گ'}
pashto_specs = {'ټ', 'څ', 'ځ', 'ډ', 'ړ', 'ږ', 'ښ', 'ګ' , 'ڼ'}
french_specs = {'É' , 'À', 'È' , 'Ù', 'Â', 'Ê', 'Î', 'Ô', 'Û', 'Ë', 'Ï', 'Ç', 'é', 'à', 'è', 'ù', 'â', 'ê', 'î', 'ô', 'û', 'ë', 'ï', 'ç' }
german_specs = {'ß', 'ä', 'ö', 'ü', 'Ä', 'Ö', 'Ü'}

In [29]:
def specs_ratio(entry, specs):
    splited = entry.split()
    words = [word for letter in specs for word in splited if letter in word]
    return len(words)/len(splited)

In [31]:
t1 = time.time()
Pers = test['Id'].apply (lambda row: specs_ratio(row, persian_specs))
Pash = test['Id'].apply (lambda row: specs_ratio(row, pashto_specs))
Germ = test['Id'].apply (lambda row: specs_ratio(row, german_specs))
Fren = test['Id'].apply (lambda row: specs_ratio(row, french_specs))
t2 = time.time()

In [51]:
t2 -t1

0.02992868423461914

In [47]:
new_preprocessed_test = tfidf_preprocessed_test.copy()

In [48]:
new_preprocessed_test['Pers'] = Pers
new_preprocessed_test['Pash'] = Pash
new_preprocessed_test['Germ'] = Germ
new_preprocessed_test['Fren'] = Fren

In [49]:
new_preprocessed_test

,0,1,2,3,4,5,6,7,8,9,...,70255,70256,70257,70258,70259,70260,Pers,Pash,Germ,Fren
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.105263
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.260870
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.144578
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.259259
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.026549,0.221239,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.035714
96,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.016393,0.000000,0.0,0.000000
97,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.180952,0.0,0.000000
98,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000


In [50]:
t1 = time.time()
Pers = train['Text'].apply (lambda row: specs_ratio(row, persian_specs))
Pash = train['Text'].apply (lambda row: specs_ratio(row, pashto_specs))
Germ = train['Text'].apply (lambda row: specs_ratio(row, german_specs))
Fren = train['Text'].apply (lambda row: specs_ratio(row, french_specs))
t2 = time.time()

In [35]:
t2 - t1

0.8392043113708496

In [36]:
new_preprocessed_X = X_tfidf.copy()

In [38]:
new_preprocessed_X['Pers'] = Pers
new_preprocessed_X['Pash'] = Pash
new_preprocessed_X['Germ'] = Germ
new_preprocessed_X['Fren'] = Fren

In [39]:
new_preprocessed_X

,0,1,2,3,4,5,6,7,8,9,...,70255,70256,70257,70258,70259,70260,Pers,Pash,Germ,Fren
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.105263
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.260870
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.144578
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.259259
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.026549,0.221239,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5465,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.058824,0.000000
5466,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.095238,0.000000
5467,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.083333,0.000000
5468,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.105263,0.000000


In [41]:
model = svm.LinearSVC(C=0.1)

In [42]:
type(new_preprocessed_X)

pandas.core.frame.DataFrame

In [43]:
model.fit(new_preprocessed_X, Y)

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:516: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  "pandas.DataFrame with sparse columns found."


LinearSVC(C=0.1)

In [51]:
pred = model.predict(new_preprocessed_test)

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:516: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  "pandas.DataFrame with sparse columns found."


In [52]:
sol2_output = pd.DataFrame({'Id' : test['Id'], 'Category' : pred})

In [53]:
sol2_output.to_csv('./sol2_output.csv', index=False)